In [221]:
import pandas as pd
import json

# 📌 Extracción

Para iniciar mi análisis, importaré los datos de la API de Telecom X. Estos datos están disponibles en formato JSON y contienen información esencial sobre los clientes, incluyendo datos demográficos, tipo de servicio contratado y estado de evasión.

✅ Cargando los datos directamente desde la API utilizando Python.

✅ Normalizando las columnas con diccionarios.

✅ Convirtiendo los datos a un DataFrame de Pandas para facilitar su manipulación.

In [222]:
with open('../data/TelecomX_Data.json','r') as f:
    data = json.loads(f.read())

In [223]:
pd_data = pd.DataFrame(data)

# 🔧 Transformación

## Conociendo el conjunto de datos

Ahora que extraí los datos, es fundamental comprender la estructura del dataset y el significado de sus columnas. Esta etapa ayudará a identificar qué variables son más relevantes para el análisis de evasión de clientes.

📌 Para facilitar este proceso, hay en el README.md un diccionario de datos con la descripción de cada columna.

¿Qué debo hacer?

✅ Explorar las columnas del dataset y verificar sus tipos de datos.

✅ Consultar el diccionario para comprender mejor el significado de las variables.

✅ Identificar las columnas más relevantes para el análisis de evasión.



In [224]:
pd_data.sample(3)

,customerID,Churn,customer,phone,internet,account
6036,8237-ULIXL,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'No', 'OnlineSecurity': 'N...","{'Contract': 'Two year', 'PaperlessBilling': '..."
3097,4315-MURBD,Yes,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
1703,2418-TPEUN,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'No', 'OnlineSecurity': 'N...","{'Contract': 'Two year', 'PaperlessBilling': '..."


## Normalizando nuestros datos

Vemos que las columnas poseen diccionarios, por lo que normalizaré para separarlos en nuevas columnas, almacenandolo en un nuevo DataFrame para poder analizar y comparar los datos.

In [225]:
data_norm = pd.json_normalize(data)

In [226]:
data_norm.sample(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
3170,4425-OWHWB,No,Female,0,No,No,20,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Credit card (automatic),19.35,433.75
5680,7762-URZQH,Yes,Male,0,Yes,No,66,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),106.05,6981.35
2048,2884-GBPFB,Yes,Female,0,Yes,No,35,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.30,2416.55


Vemos los tipos de datos almacenados en las columnas

In [227]:
data_norm.dtypes

customerID                    object
Churn                         object
customer.gender               object
customer.SeniorCitizen         int64
customer.Partner              object
customer.Dependents           object
customer.tenure                int64
phone.PhoneService            object
phone.MultipleLines           object
internet.InternetService      object
internet.OnlineSecurity       object
internet.OnlineBackup         object
internet.DeviceProtection     object
internet.TechSupport          object
internet.StreamingTV          object
internet.StreamingMovies      object
account.Contract              object
account.PaperlessBilling      object
account.PaymentMethod         object
account.Charges.Monthly      float64
account.Charges.Total         object
dtype: object

Observamos que la columna **account.Charges.Total** tiene datos de tipo *object*, entonces la convertimos a tipo *float64*.

In [228]:
data_norm['account.Charges.Total'] = pd.to_numeric(data_norm['account.Charges.Total'], errors='coerce')

In [229]:
data_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

Observamos que nuestro df es de 7267 filas por 21 columnas.

In [230]:
data_norm.shape

(7267, 21)

En total tenemos 152607 datos

In [231]:
data_norm.size

152607

Vemos una lista de las columnas.

In [232]:
data_norm.columns

Index(['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen',
       'customer.Partner', 'customer.Dependents', 'customer.tenure',
       'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService',
       'internet.OnlineSecurity', 'internet.OnlineBackup',
       'internet.DeviceProtection', 'internet.TechSupport',
       'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract',
       'account.PaperlessBilling', 'account.PaymentMethod',
       'account.Charges.Monthly', 'account.Charges.Total'],
      dtype='object')

## Comprobación de incoherencias en los datos

En este paso, verifico si hay problemas en los datos que puedan afectar el análisis. Prestando atención a valores ausentes, duplicados, errores de formato e inconsistencias en las categorías. Este proceso es esencial para asegurarme de que los datos estén listos para las siguientes etapas.

In [233]:
data_norm.describe()

,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly,account.Charges.Total
count,7267.000000,7267.000000,7267.000000,7256.000000
mean,0.162653,32.346498,64.720098,2280.634213
std,0.369074,24.571773,30.129572,2268.632997
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.425000,400.225000
50%,0.000000,29.000000,70.300000,1391.000000
75%,0.000000,55.000000,89.875000,3785.300000
max,1.000000,72.000000,118.750000,8684.800000


Vemos que 'Churn' tiene 3 valores posibles, cuando sólo deberían ser 2.

In [234]:
data_norm.nunique()

customerID                   7267
Churn                           3
customer.gender                 2
customer.SeniorCitizen          2
customer.Partner                2
customer.Dependents             2
customer.tenure                73
phone.PhoneService              2
phone.MultipleLines             3
internet.InternetService        4
internet.OnlineSecurity         3
internet.OnlineBackup           3
internet.DeviceProtection       3
internet.TechSupport            3
internet.StreamingTV            3
internet.StreamingMovies        3
account.Contract                3
account.PaperlessBilling        2
account.PaymentMethod           4
account.Charges.Monthly      1585
account.Charges.Total        6530
dtype: int64

Vemos los valores únicos por columna para corroborar valores

In [235]:
for columna in data_norm.columns:
    print(f'{columna} \t {data_norm[columna].unique()}')

customerID 	 ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn 	 ['No' 'Yes' '']
customer.gender 	 ['Female' 'Male']
customer.SeniorCitizen 	 [0 1]
customer.Partner 	 ['Yes' 'No']
customer.Dependents 	 ['Yes' 'No']
customer.tenure 	 [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
phone.PhoneService 	 ['Yes' 'No']
phone.MultipleLines 	 ['No' 'Yes' 'No phone service']
internet.InternetService 	 ['DSL' 'Fiber optic' 'No' 'Fibjsoner optic']
internet.OnlineSecurity 	 ['No' 'Yes' 'No internet service']
internet.OnlineBackup 	 ['Yes' 'No' 'No internet service']
internet.DeviceProtection 	 ['No' 'Yes' 'No internet service']
internet.TechSupport 	 ['Yes' 'No' 'No internet service']
internet.StreamingTV 	 ['Yes' 'No' 'No internet service']
internet.StreamingMovies 	 ['No' 'Yes' 'No 

⚠️ En 'Churn' no debería haber valores vacíos.

Verificamos duplicados de filas

In [236]:
sum(int(x) for x in data_norm.duplicated().values)

0

No hay filas duplicadas

Verificamos si hay un sólo registro para cada cliente

In [237]:
len(data_norm['customerID'].unique()) == len(data_norm)

True

No hay más de un registro para un cliente

## Manejo de Inconsistencias

Aplico las correcciones necesarias. Ajusto los datos para asegurarme de que estén completos y coherentes, preparándolos para las siguientes etapas del análisis.

In [238]:
data_norm.sample(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
7054,9715-SBVSU,No,Male,0,Yes,Yes,14,Yes,No,DSL,...,Yes,No,No,No,Yes,Two year,Yes,Bank transfer (automatic),61.40,815.55
1803,2569-WGERO,No,Female,0,No,No,72,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.40
2924,4065-JJAVA,No,Female,0,No,No,1,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,No,Electronic check,49.50,49.50


Buscamos la fila con 'Churn' == ''

In [239]:
data_norm[data_norm['Churn']=='']

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
30,0047-ZHDTW,,Female,0,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.30
75,0120-YZLQA,,Male,0,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.10
96,0154-QYHJU,,Male,0,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.20
98,0162-RZGMZ,,Female,1,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,1,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,0,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.20
7180,9872-RZQQB,,Female,0,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,0,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,0,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.90


Eliminaremos esos 224 registros

Obtenemos una lista con sus índices

In [240]:
filas_a_eliminar = data_norm[data_norm['Churn']==''].index

In [241]:
data_norm.drop(filas_a_eliminar, inplace=True)
data_norm.reset_index(drop=True, inplace=True)

Verifico si no quedaron registros con 'Churn' == '', y si la cantidad restante es correcta

In [242]:
( len(data_norm[data_norm['Churn']==''].index) == 0 ) & ( len(data_norm) == ( len(pd_data) - len(filas_a_eliminar) ) )

True

In [243]:
data_norm

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90
7039,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70
7040,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7041,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


## Columna de cuentas diarias

Ahora que los datos están limpios, creamos la columna "Cuentas_Diarias" solicitada por el desafío, bajo el nombre "account.Charges.Daily". Usamos la facturación mensual para calcular el valor diario, proporcionando una visión más detallada del comportamiento de los clientes a lo largo del tiempo.

In [249]:
data_norm['account.Charges.Daily'] = (data_norm['account.Charges.Monthly'] / 30).round(2)
data_norm.head(3)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.46


## Estandarización y Transformación de Datos

La estandarización y transformación de datos es una etapa opcional, pero altamente recomendada, ya que busca hacer que la información sea más consistente, comprensible y adecuada para el análisis. Durante esta fase, por ejemplo, podemos convertir valores textuales como "Sí" y "No" en valores binarios (1 y 0), lo que facilita el procesamiento matemático y la aplicación de modelos analíticos.

Además, traducir o renombrar columnas y datos hace que la información sea más accesible y fácil de entender, especialmente cuando se trabaja con fuentes externas o términos técnicos. Aunque no es un paso obligatorio, puede mejorar significativamente la claridad y comunicación de los resultados, facilitando la interpretación y evitando confusiones, especialmente al compartir información con stakeholders no técnicos.

Los nombres de las columnas decido dejarlos en inglés, me resulta más compacto a la vista, que su correspondiente al español.

Pero sí modificaré las columnas:
```
{
    'Churn': {'No': 0, 'Yes': 1},
    'customer.gender': {'Male': 0, 'Female': 1},
    'customer.Partner': {'No': 0, 'Yes': 1},
    'customer.Dependents': {'No': 0, 'Yes': 1},
    'phone.PhoneService': {'No': 0, 'Yes': 1},
    'phone.MultipleLines': {'No': 0, 'Yes': 1, 'No phone service': 2},
    'internet.InternetService': {'No': 0, 'DSL': 1, 'Fiber optic': 2, 'Fibjsoner optic': 3},
    'internet.OnlineSecurity': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.OnlineBackup': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.DeviceProtection': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.TechSupport': : {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.StreamingTV': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.StreamingMovies': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'account.Contract': {'One year': 12, 'Month-to-month': 1, 'Two year': 24},
    'account.PaperlessBilling':	{'No': 0, 'Yes': 1},
    'account.PaymentMethod': {'Mailed check': 0, 'Electronic check': 1, 'Credit card (automatic)': 2, 'Bank transfer (automatic)': 3}
}
```


In [258]:
diccionario_reemplazo_multiple = {
    'Churn': {'No': 0, 'Yes': 1},
    'customer.gender': {'Male': 0, 'Female': 1},
    'customer.Partner': {'No': 0, 'Yes': 1},
    'customer.Dependents': {'No': 0, 'Yes': 1},
    'phone.PhoneService': {'No': 0, 'Yes': 1},
    'phone.MultipleLines': {'No': 0, 'Yes': 1, 'No phone service': 2},
    'internet.InternetService': {'No': 0, 'DSL': 1, 'Fiber optic': 2, 'Fibjsoner optic': 3},
    'internet.OnlineSecurity': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.OnlineBackup': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.DeviceProtection': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.TechSupport': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.StreamingTV': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'internet.StreamingMovies': {'No': 0, 'Yes': 1, 'No internet service': 2},
    'account.Contract': {'One year': 12, 'Month-to-month': 1, 'Two year': 24},
    'account.PaperlessBilling':	{'No': 0, 'Yes': 1},
    'account.PaymentMethod': {'Mailed check': 0, 'Electronic check': 1, 'Credit card (automatic)': 2, 'Bank transfer (automatic)': 3}
}


In [259]:
data_norm = data_norm.replace(diccionario_reemplazo_multiple)
data_norm.sample(3)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11312\1891672484.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_norm = data_norm.replace(diccionario_reemplazo_multiple)


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,account.Charges.Daily
5339,7542-CYDDM,0,0,0,0,0,18,1,0,0,...,2,2,2,2,1,0,0,20.05,358.50,0.67
436,0637-YLETY,1,1,0,0,0,16,1,1,2,...,1,0,0,1,1,1,2,95.60,1555.65,3.19
2623,3733-LSYCE,0,1,0,1,0,15,1,0,2,...,0,0,0,0,1,0,3,75.35,1114.55,2.51


#📊 Carga y análisis

#📄Informe final